# **10장. 자연어 처리를 위한 임베딩**

임베딩 : 자연어(사람이 사용하는 언어) -> 벡터(숫자, 컴퓨터가 이해할 수 있는 언어) 형태로 변환하는 결과 또는 과정

임베딩의 역할

- 단어 및 문장 간 관련성 계산
- 의미적 혹은 문법적 정보 함축(ex: 왕-여왕, 교사-학생)

**10.1.1 희소 표현 기반 임베딩**

희소 표현(sparse representation) : 대부분의 값이 0으로 채워져 있는 경우.

- 원 핫 인코딩(one-hot encoding) : 단어 N개를 각각 N차원 벡터로 표현하는 방식, 단어가 포함되는 위치에 1을 넣고 나머지를 0으로 채움

In [1]:
#원-핫 인코딩 적용
import pandas as pd
class2 = pd.read_csv('/content/drive/MyDrive/pytorch_ex/chap10/data/class2.csv')

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

**10.1.2 횟수 기반 임베딩**

횟수 기반 : 단어가 출현할 빈도를 고려하여 임베딩하는 방법

- 카운터 벡터(counter vector) : 문서 집합에서 단어를 토큰으로 생성, 각 단어의 출현 빈도수를 이용하여 인코딩해 벡터를 만드는 방법

  즉, 토크나이징과 벡터화가 동시에 가능한 방법.

- TF-IDF(Term Frequency-Inverse Document Frequency)

  TF: 한 문서 내에서 특정 단어의 출현 빈도 -> 높을수록 해당 문서와 단어의 관련이 높음

  DF: 전체 문서 중 특정 단어가 포함된 문서의 개수 -> 높을수록 일반적인 단어로 간주되어 가중치를 낮추어줘야 함

  IDF: DF값이 클수록 가중치값을 낮춰주기 위해 DF 값에 역수를 취한 것

In [2]:
#코퍼스(말뭉치)에 카운터 벡터 적용

from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is last chance.',
    'and if you do not have this chance',
    'you will never get any chance',
    'will you do get this one?',
    'please, get this chance'
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [3]:
#적용 결과를 배열로 변환
vect.transform(['you will never get any chance']).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]])

In [4]:
#불용어를 제거한 카운터 벡터
vect = CountVectorizer(stop_words=['and', 'is', 'please', 'this']).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

In [7]:
#TF-IDF 적용 후 행렬로 표현
from sklearn.feature_extraction.text import TfidfVectorizer
doc = [ 'I like machine learning', 'I ove deep learning', 'I run everyday' ]
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)
print('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), '행렬을 만들었습니다')
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


**10.1.3 예측 기반 임베딩**

: 신경망 구조나 모델을 이용하여, 특정 문맥에서 어떤 단어가 나올지 예측하면서 단어를 벡터로 만드는 방식

- 워드투벡터(Word2Vec) : 신경망 알고리즘. 텍스트의 각 단어를 하나의 일련의 벡터로 출력하되, 의미론적으로 유사한 단어의 벡터를 서로 가깝게 표현
- CBOW(Continuous Bag Of Words) : 주변 단어에서 중심 단어 예측
- skip-gram : 중심 단어에서 주변 단어 예측

In [9]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 7.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requ

In [10]:
#데이터셋을 메모리로 로딩, 토큰화 적용

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import gensim
from gensim.models import Word2Vec

sample = open('/content/drive/MyDrive/pytorch_ex/chap10/data/peter.txt', 'r', encoding='UTF8')
s = sample.read()

f = s.replace('\n', ' ') #줄바꿈 -> 공백으로 변환
data = []

for i in sent_tokenize(f) : #각 문장마다~
  temp =[]
  for j in word_tokenize(i) : #문장을 단어로 토큰화하여, 각 단어마다~
      temp.append(j.lower()) #소문자로 변환하여 저장
  data.append(temp)

print(data)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject